In [1]:
from pycbc import distributions
from pycbc.waveform import get_td_waveform
import matplotlib.pyplot as plt
import numpy as np
import pylab
from tqdm.notebook import tqdm
from gwpy.timeseries import TimeSeries
import pandas as pd
import os
import csv

In [2]:

print("Generated Binary Mass Distributions for BBH")

bbh_two_mass_distributions = distributions.Uniform(mass1=(10, 50),
                                               mass2=(10, 50))

bbh_two_mass_samples = bbh_two_mass_distributions.rvs(size=5000)

Generated Binary Mass Distributions for BBH


In [ ]:
# # Generate some noise with an advanced ligo psd
# flow = 5.0
# delta_f = 1.0 / 16
# flen = int(2048 / delta_f) + 1
# psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

# # Generate 1 seconds of noise at 4096 Hz
# delta_t = 1.0 / 4096
# tsamples = int(4 / delta_t) ############################################# changed this from int(1 / delta_t) to generate 4 seconds of noise 
# ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd, seed=127)

# ts = pycbc.filter.highpass(ts, 30.0)
# ts = pycbc.filter.lowpass_fir(ts, 1000.0, 8)

# # noise = ts * 1e3
# noise = ts * 1e4

In [3]:
print("Using Binary Mass Distributions to generate BBH waveforms")
print("Generating BBH Waveforms, Noise and Merging them together and writing into disk, it may take few minutes...")

for i in tqdm(range(len(bbh_two_mass_samples))):

    hp, hc = get_td_waveform(approximant="SEOBNRv2",                                # what are hp and hc?
                         mass1=bbh_two_mass_samples[i][0],
                         mass2=bbh_two_mass_samples[i][1],
                         delta_t=1.0/4096,
                         f_lower=40)                                                # change to 40 Hz


    # Generate some noise with an advanced ligo psd
    flow = 5.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 1 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t) ############################################# changed this from int(1 / delta_t) to generate 4 seconds of noise 
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    ts = pycbc.filter.highpass(ts, 30.0)
    ts = pycbc.filter.lowpass_fir(ts, 1000.0, 8)

    # ts *= 1e3
    ts *= 1e4
    noise = TimeSeries.from_pycbc(ts)

    # st = np.random.randint(0,2)
    
    st = 1

    signal = TimeSeries.from_pycbc(hp)
    
    signal.t0 = st
    data = noise.inject(signal)
    
    data.write("data/merged/bbh_merged_noise_signal_"+str(i)+".txt")

Using Binary Mass Distributions to generate BBH waveforms
Generating BBH Waveforms, Noise and Merging them together and writing into disk, it may take few minutes...


  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
# MERGING NOISE + SIgnal Templates into single file


#for i in tqdm(range(5)):
path = "data/merged/"
files= os.listdir(path)
f = open('data/Final_BBH_Merged_Noise_Signal.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
for i in range(5000):

    # Generate some noise with an advanced ligo psd
    flow = 5.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 1 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t) ############################################# changed this from int(1 / delta_t) to generate 4 seconds of noise 
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    ts = pycbc.filter.highpass(ts, 30.0)
    ts = pycbc.filter.lowpass_fir(ts, 1000.0, 8)

    # ts *= 1e3
    ts *= 1e4
    noise = TimeSeries.from_pycbc(ts)
    noise.write("data/noise/noise_"+str(i)+".txt")

In [ ]:
# MERGING NOISE Templates into single file

path_1 = "data/noise/"
files_1= os.listdir(path_1)
f1 = open('data/Final_Merged_Noise_Reduced_No_Abs.csv', 'w')
cw_1 = csv.writer(f1)

for i in tqdm(files_1):
    #print(files)
    df = pd.read_csv(path_1+i,sep = ' ', header=None)
    c = df[:][1]
    cw_1.writerow(c)
f1.close()

  0%|          | 0/5000 [00:00<?, ?it/s]